In [ ]:
# PLANTILLA CREWAI - SISTEMA MULTIAGENTE
# ====================================

from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, FileReadTool
import os

# PASO 1: Configuración de variables de entorno
# ============================================
os.environ["OPENAI_API_KEY"] = "tu-api-key-aquí"
os.environ["SERPER_API_KEY"] = "tu-serper-api-key-aquí"  # Para búsquedas web

# PASO 2: Configuración de herramientas
# ====================================
# SerperDevTool: Para búsquedas en internet
search_tool = SerperDevTool()

# FileReadTool: Para leer archivos
file_tool = FileReadTool()

# PASO 3: Definición de agentes
# ============================

# Agente 1: Investigador - Se encarga de buscar información
investigador = Agent(
    role='Investigador Senior',
    goal='Encontrar y analizar información relevante sobre el tema solicitado',
    backstory="""Eres un investigador experimentado con habilidades excepcionales 
                 para encontrar información precisa y relevante. Te especializas en 
                 analizar múltiples fuentes y extraer insights valiosos.""",
    verbose=True,  # Muestra el proceso de pensamiento
    allow_delegation=True,  # Puede delegar tareas a otros agentes
    tools=[search_tool, file_tool],
    max_iter=3,  # Máximo 3 iteraciones por tarea
    memory=True  # Mantiene memoria de interacciones anteriores
)

# Agente 2: Analista - Procesa y estructura la información
analista = Agent(
    role='Analista de Datos',
    goal='Analizar la información recopilada y crear insights estructurados',
    backstory="""Eres un analista experto que puede procesar grandes cantidades 
                 de información y convertirla en insights claros y accionables. 
                 Te especializas en encontrar patrones y conexiones.""",
    verbose=True,
    allow_delegation=False,  # No delega, es especialista
    tools=[],  # No necesita herramientas externas
    max_iter=2
)

# Agente 3: Redactor - Crea el contenido final
redactor = Agent(
    role='Redactor Técnico',
    goal='Crear documentos bien estructurados y fáciles de entender',
    backstory="""Eres un redactor técnico experto que puede tomar información 
                 compleja y convertirla en contenido claro, bien estructurado 
                 y fácil de entender para diferentes audiencias.""",
    verbose=True,
    allow_delegation=False,
    tools=[],
    max_iter=2
)

# PASO 4: Definición de tareas
# ===========================

# Tarea 1: Investigación
tarea_investigacion = Task(
    description="""
    Investiga a fondo sobre {tema} y recopila información de al menos 3 fuentes confiables.
    
    Debes incluir:
    - Definición y conceptos clave
    - Tendencias actuales
    - Casos de uso principales
    - Desafíos y limitaciones
    
    Entrega un resumen estructurado con las fuentes utilizadas.
    """,
    expected_output="Un informe de investigación con información detallada y fuentes citadas",
    agent=investigador,
    tools=[search_tool]
)

# Tarea 2: Análisis
tarea_analisis = Task(
    description="""
    Analiza la información recopilada por el investigador sobre {tema}.
    
    Debes:
    - Identificar los puntos clave más importantes
    - Encontrar conexiones y patrones
    - Evaluar la relevancia de cada insight
    - Crear una estructura lógica para presentar la información
    
    Entrega un análisis estructurado con recomendaciones.
    """,
    expected_output="Un análisis detallado con insights clave y recomendaciones estructuradas",
    agent=analista,
    context=[tarea_investigacion]  # Depende de la tarea anterior
)

# Tarea 3: Redacción final
tarea_redaccion = Task(
    description="""
    Crea un documento final sobre {tema} basado en la investigación y análisis previos.
    
    El documento debe incluir:
    - Introducción clara
    - Desarrollo estructurado en secciones
    - Conclusiones y recomendaciones
    - Formato profesional y fácil de leer
    
    Adapta el lenguaje para una audiencia técnica pero accesible.
    """,
    expected_output="Un documento final bien estructurado, claro y profesional",
    agent=redactor,
    context=[tarea_investigacion, tarea_analisis],  # Depende de ambas tareas anteriores
    output_file="resultado_final.md"  # Guarda el resultado en archivo
)

# PASO 5: Configuración del equipo (Crew)
# =======================================

crew = Crew(
    agents=[investigador, analista, redactor],  # Lista de agentes
    tasks=[tarea_investigacion, tarea_analisis, tarea_redaccion],  # Lista de tareas
    process=Process.sequential,  # Proceso secuencial (una tarea tras otra)
    verbose=2,  # Nivel de detalle en logs (0=mínimo, 2=máximo)
    memory=True,  # Habilita memoria compartida entre agentes
    cache=True,  # Habilita cache para optimizar rendimiento
    max_rpm=10,  # Máximo 10 requests por minuto
    share_crew=False  # No comparte con CrewAI cloud
)

# PASO 6: Función principal para ejecutar
# ======================================

def ejecutar_crew(tema_investigacion):
    """
    Ejecuta el crew con un tema específico
    
    Args:
        tema_investigacion (str): El tema sobre el cual investigar
        
    Returns:
        dict: Resultado de la ejecución
    """
    try:
        print(f"🚀 Iniciando investigación sobre: {tema_investigacion}")
        
        # Ejecuta el crew con el tema proporcionado
        resultado = crew.kickoff(
            inputs={"tema": tema_investigacion}
        )
        
        print("✅ Crew ejecutado exitosamente!")
        return resultado
        
    except Exception as e:
        print(f"❌ Error durante la ejecución: {str(e)}")
        return None

# PASO 7: Ejemplo de uso
# =====================

if __name__ == "__main__":
    # Tema de ejemplo
    tema = "Inteligencia Artificial en el sector salud"
    
    # Ejecutar el crew
    resultado = ejecutar_crew(tema)
    
    # Mostrar resultado
    if resultado:
        print("\n" + "="*50)
        print("RESULTADO FINAL:")
        print("="*50)
        print(resultado)
        
        # El archivo resultado_final.md se habrá creado automáticamente

# PASO 8: Funciones adicionales útiles
# ===================================

def agregar_agente_personalizado(rol, objetivo, historia, herramientas=[]):
    """
    Función helper para crear agentes personalizados fácilmente
    """
    return Agent(
        role=rol,
        goal=objetivo,
        backstory=historia,
        verbose=True,
        tools=herramientas,
        max_iter=3
    )

def crear_tarea_personalizada(descripcion, resultado_esperado, agente, contexto=None):
    """
    Función helper para crear tareas personalizadas
    """
    return Task(
        description=descripcion,
        expected_output=resultado_esperado,
        agent=agente,
        context=contexto if contexto else []
    )

# CONFIGURACIONES ADICIONALES
# ==========================

# Para usar diferentes LLMs:
# from langchain.llms import Ollama
# crew.llm = Ollama(model="llama2")

# Para procesos paralelos en lugar de secuenciales:
# process=Process.hierarchical

# Para callbacks personalizados:
# def mi_callback(step):
#     print(f"Paso completado: {step}")
# crew.step_callback = mi_callback